In [ ]:
!pip install -q pylatexenc
!pip install -q qiskit
!pip install -q qiskit_aer

<img src="/content/Deutsch-Jozsa.png" />

In [ ]:
from qiskit import QuantumCircuit
import numpy as np

def dj_function(num_qubits):
    """
    Create a random Deutsch-Jozsa function.
    """

    qc = QuantumCircuit(num_qubits + 1)
    if np.random.randint(0, 2):
        # Flip output qubit with 50% chance
        qc.x(num_qubits)
    if np.random.randint(0, 2):
        # return constant circuit with 50% chance
        return qc

    # next, choose half the possible input states
    on_states = np.random.choice(
        range(2**num_qubits),  # numbers to sample from
        2**num_qubits // 2,  # number of samples
        replace=False,  # makes sure states are only sampled once
    )

    def add_cx(qc, bit_string):
        for qubit, bit in enumerate(reversed(bit_string)):
            if bit == "1":
                qc.x(qubit)
        return qc

    for state in on_states:
        qc.barrier()  # Barriers are added to help visualize how the functions are created. They can safely be removed.
        qc = add_cx(qc, f"{state:0b}")
        qc.mcx(list(range(num_qubits)), num_qubits)
        qc = add_cx(qc, f"{state:0b}")

    qc.barrier()

    return qc

In [ ]:
display(dj_function(1).draw())

q_0: 
     
q_1:

In [ ]:
display(dj_function(3).draw())

q_0: 
     
q_1: 
     
q_2: 
     
q_3:

In [ ]:
def compile_circuit(function: QuantumCircuit):
    """
    Compiles a circuit for use in the Deutsch-Jozsa algorithm.
    """
    n = function.num_qubits - 1
    qc = QuantumCircuit(n + 1, n)
    qc.x(n)
    qc.h(range(n + 1))
    qc.compose(function, inplace=True)
    qc.h(range(n))
    qc.measure(range(n), range(n))
    return qc

In [ ]:
from qiskit_aer import AerSimulator

def dj_algorithm(function: QuantumCircuit):
    """
    Determine if a Deutsch-Jozsa function is constant or balanced.
    """
    qc = compile_circuit(function)

    print('Cały obwód')
    display(qc.draw())

    result = AerSimulator().run(qc, shots=1, memory=True).result()
    measurements = result.get_memory()
    if "1" in measurements[0]:
        return "balanced"
    return "constant"

In [ ]:
f = dj_function(3)
display(f.draw())
display(dj_algorithm(f))

░ ┌───┐     ┌───┐ ░                 ░ ┌───┐     ┌───┐ ░           »
q_0: ──────░─┤ X ├──■──┤ X ├─░────────■────────░─┤ X ├──■──┤ X ├─░────────■──»
           ░ ├───┤  │  ├───┤ ░ ┌───┐  │  ┌───┐ ░ ├───┤  │  ├───┤ ░        │  »
q_1: ──────░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░────────■──»
           ░ ├───┤  │  ├───┤ ░ ├───┤  │  ├───┤ ░ └───┘  │  └───┘ ░ ┌───┐  │  »
q_2: ──────░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░────────■────────░─┤ X ├──■──»
     ┌───┐ ░ └───┘┌─┴─┐└───┘ ░ └───┘┌─┴─┐└───┘ ░      ┌─┴─┐      ░ └───┘┌─┴─┐»
q_3: ┤ X ├─░──────┤ X ├──────░──────┤ X ├──────░──────┤ X ├──────░──────┤ X ├»
     └───┘ ░      └───┘      ░      └───┘      ░      └───┘      ░      └───┘»
«           ░ 
«q_0: ──────░─
«           ░ 
«q_1: ──────░─
«     ┌───┐ ░ 
«q_2: ┤ X ├─░─
«     └───┘ ░ 
«q_3: ──────░─
«           ░

Cały obwód


┌───┐           ░ ┌───┐     ┌───┐ ░                 ░ ┌───┐     ┌───┐ ░ »
q_0: ┤ H ├───────────░─┤ X ├──■──┤ X ├─░────────■────────░─┤ X ├──■──┤ X ├─░─»
     ├───┤           ░ ├───┤  │  ├───┤ ░ ┌───┐  │  ┌───┐ ░ ├───┤  │  ├───┤ ░ »
q_1: ┤ H ├───────────░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─»
     ├───┤           ░ ├───┤  │  ├───┤ ░ ├───┤  │  ├───┤ ░ └───┘  │  └───┘ ░ »
q_2: ┤ H ├───────────░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░────────■────────░─»
     ├───┤┌───┐┌───┐ ░ └───┘┌─┴─┐└───┘ ░ └───┘┌─┴─┐└───┘ ░      ┌─┴─┐      ░ »
q_3: ┤ X ├┤ H ├┤ X ├─░──────┤ X ├──────░──────┤ X ├──────░──────┤ X ├──────░─»
     └───┘└───┘└───┘ ░      └───┘      ░      └───┘      ░      └───┘      ░ »
c: 3/════════════════════════════════════════════════════════════════════════»
                                                                             »
«                     ░ ┌───┐┌─┐      
«q_0: ───────■────────░─┤ H ├┤M├──────
«            │        ░ ├───┤└╥┘┌─┐   
«q_1: ───────■────────░─┤ H ├─╫─┤M├───
«     ┌───┐  │  ┌───┐ ░ ├───┤ ║ └╥┘┌─┐
«q_2: ┤ X ├──■──┤ X ├─░─┤ H ├─╫──╫─┤M├
«     └───┘┌─┴─┐└───┘ ░ └───┘ ║  ║ └╥┘
«q_3: ─────┤ X ├──────░───────╫──╫──╫─
«          └───┘      ░       ║  ║  ║ 
«c: 3/════════════════════════╩══╩══╩═
«                             0  1  2

'balanced'